In [164]:
import math
import numpy as np
import pandas as pd
from scipy.optimize import linprog


T = 20000  # number of tests in stock

# Test/dev df
df = pd.DataFrame.from_dict(orient='index',
                            columns=['population', 'capacity', 'infected'],
                            data={
    'Bucuresti': [70000, 600, 300],
    'Constanta': [20000, 200, 50],
    'Craiova'  : [10000, 100, 10]
})

df.loc['Total'] = df.sum(axis=0)
df.head()

,population,capacity,infected
Bucuresti,70000,600,300
Constanta,20000,200,50
Craiova,10000,100,10
Total,100000,900,360


In [176]:
# def objective(q, p, b, D):
#     '''
#     The objective function of optimization to be maximized.
    
#     :param q - vector of tests/day (assumed stationary)
#     :param p - vector of population per region
#     :param b - vector of infected count per region
#     :parma D - number of days to consider
    
#     :return float
#     '''
#     assert q.shape == p.shape == b.shape
    
#     coef = coefficients(p, b, D)
#     return np.dot(q, coef)

Defining a linear optimization problem.

* b - diseased per region
* p - population per region
* D - number of days considered
* x - decision variables (tests/day)
* T - number of tests available
* cap - capacity of region (tests/day)

$$c=D(b .* 1/p)$$

$$\max_{x}\quad c^Tx$$

$$c^Tx \le T$$

$$0 \le x \le cap$$

In [191]:
def coefficients(p, b, D):
    '''
    Get c vector (in LP notation).
    
    :param p - vector of population per region
    :param b - vector of infected count per region
    :param D - number of days to consider
    
    :return np.array
    '''
    return b/p


def arrange_bounds(cap):
    '''
    Arrange the bounds in appropriate form.
    
    :param cap - capacity vector.
    
    :return list - [(0, cap1), (0, cap2), ...]
    '''
    bounds = []
    for c in cap:
        bounds.append((0, c))
    return bounds


def optimize(df, T, D = None):
    '''
    Calculate the number of tests per day per region.
    
    :param df - dataframe with columns ('population', 'capacity', 'infected') and
                rows ('region1', ..., 'Total')
    :param T  - int or list, specifies the nr of available tests.
    :param D  - None, int, list, specifies the nr of days considered.
    
    :return df
    '''
    
    if (D is None and type(T) == int):
        D = math.ceil(T/df['capacity']['Total'])  # ceiling
    
    elif (type(D) == int and type(T) == int):
        # Base case
        # continue the usual algorithm
        pass
    
    elif (type(D) == list or type(T) == list):
        # Recursive case
        
        ### Manage types
        if (type(T) == int):
            T_list = [T]
        elif (type(T) == list):
            T_list = T[:]
        else:
            raise ValueError("Invalid type for T.")
        
        if (type(D) == int):
            D_list = [D]
        elif (type(D) == list):
            D_list = D[:]
        else:
            raise ValueError("Invalid type for D.")
        ###
        
        for t in T_list:
            for d in D_list:
                col = optimize(df, t, d).iloc[:, -1]
                df = pd.concat([df, col], axis=1)
        
        return df
    
    else:
        raise ValueError("Invalid type for D.")
    
    df_cl = df.drop(['Total'], axis=0)
    
    p = df_cl['population'].to_numpy()
    b = df_cl['infected'].to_numpy()
    c = df_cl['capacity'].to_numpy()
    C_tot = df['capacity']['Total']
    
    coef = (-1)*coefficients(p, b, D)  # negate to maximize
    A_ub = D*np.ones(shape=(1, coef.shape[0]))  # to matrix
    b_ub = np.array([T]) 
    bounds = arrange_bounds(c)

    res = linprog(coef, A_ub=A_ub, b_ub=b_ub, bounds=bounds)
    
    if res['success']:
        df_cl['Resources=({} tests, {} days). Units=(total tests in {} days) Max_tests={}'.format(T, D, D, C_tot*D)] = D*res['x'].astype(np.int32)
    else:
        print(res['message'])
        df_cl['Resources=({} tests, {} days). Units=(total tests in {} days)'.format(T, D, D)] = np.nan
        
    df_cl.loc['Total'] = df_cl.sum(axis=0)
    
    return df_cl

In [192]:
df_opt = optimize(df, T=[5000, 10000, 15000], D=[15,20])
df_opt

,population,capacity,infected,"Resources=(5000 tests, 15 days). Units=(total tests in 15 days) Max_tests=13500","Resources=(5000 tests, 20 days). Units=(total tests in 20 days) Max_tests=18000","Resources=(10000 tests, 15 days). Units=(total tests in 15 days) Max_tests=13500","Resources=(10000 tests, 20 days). Units=(total tests in 20 days) Max_tests=18000","Resources=(15000 tests, 15 days). Units=(total tests in 15 days) Max_tests=13500","Resources=(15000 tests, 20 days). Units=(total tests in 20 days) Max_tests=18000"
Bucuresti,70000,600,300,4995,4980,8985,9980,8985,11980
Constanta,20000,200,50,0,0,990,0,2985,2980
Craiova,10000,100,10,0,0,0,0,1485,0
Total,100000,900,360,4995,4980,9975,9980,13455,14960


In [193]:
# dev
# from scipy.optimize import linprog

# T=1
# # Define the LP problem.
# coef = (-1)*coefficients(p, b, D)  # negate to maximize
# A_ub = D*np.ones(shape=(1, coef.shape[0]))  # to matrix
# b_ub = np.array([T]) 
# bounds = arrange_bounds(c)


# res = linprog(coef, A_ub=A_ub, b_ub=b_ub, bounds=bounds)
# res